이 문제를 해결하는 방법에는 두 가지가 있습니다.

- 공개 레이블이 지정된 데이터 세트 사용
- 라벨이 지정된 맞춤 데이터 세트 만들기

이 자습서에서는 처음부터 자체 데이터 세트를 생성합니다.

먼저 데이터 세트의 이미지를 수집합니다. 이 단계는 이미 완료되었다고 가정합니다.

이제 이미지에 레이블을 지정해야합니다. 많은 인기있는 라벨링 도구가 있으며 LabelIMG를 사용할 것입니다.

LabelIMG를 설치하려면 다음 코드를 실행하십시오 (Colab은 GUI 응용 프로그램을 지원하지 않으므로 로컬 터미널에서 수행하십시오).

```
pip install labelImg
```
```
labelImg imagesdir
```
![](https://ichi.pro/assets/images/max/724/1*6iL6pI9bVRV9qa8RvpzkEA.png)

예를 들어 자동차 및 자전거와 같은 두 가지 클래스를 사용하여 다음과 같이 메모장에 레이블 맵 (label_map.pbtxt)을 만듭니다.

```
item {
    id: 1
    name: 'car'
}

item {
    id: 2
    name: 'bike'
}
```



# json 병합

https://dacon.io/competitions/official/235644/codeshare/1705?page=1&dtype=recent



In [ ]:
import os
label_path = '/content/drive/Shareddrives/OD/data_real/train_etc/labels/'
files = os.listdir(label_path)
print('json 파일 총 개수 :', len(files))

json 파일 총 개수 : 1748


In [ ]:
import json

all_json = None

for file in files :
    json_data = json.load(open(label_path + file))

    if all_json == None: 
        all_json = json_data
    else: 
        all_json['features'].extend(json_data['features'])

In [ ]:
with open('/content/drive/Shareddrives/OD/data_real/train_etc/labels/labels.json', 'w') as outfile:
    json.dump(all_json, outfile, indent='\t')

label.json 파일을 이미지 폴더로 이동

다음을 수행 할 수 있습니다.

- 직접 TFRecord 만들기
- Roboflow에 주석을 업로드하고 TFRecord 형식으로 데이터 세트를 가져옵니다.

참고로 여기에 TFRecord를 수동으로 생성하는 샘플 .py 스크립트가 있습니다.

```python
import pandas as pd
import numpy as np
import csv
import re
import cv2 
import os
import glob
import xml.etree.ElementTree as ET

import io
import tensorflow as tf
from collections import namedtuple, OrderedDict

import shutil
import urllib.request
import tarfile
import argparse

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow.compat.v1 as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple
xml_dir = 'images/test'
image_dir = 'images/test'

label_map = label_map_util.load_labelmap('annotations/label_map.pbtxt')
label_map_dict = label_map_util.get_label_map_dict(label_map)

output_path = 'annotations/test.record'

def xml_to_csv(path):
    """Iterates through all .xml files (generated by labelImg) in a given directory and combines
    them in a single Pandas dataframe.

    Parameters:
    ----------
    path : str
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """

    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def class_text_to_int(row_label):
    return label_map_dict[row_label]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

csv_path = None
def main(_):

    writer = tf.python_io.TFRecordWriter(output_path)
    path = os.path.join(image_dir)
    examples = xml_to_csv(xml_dir)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file: {}'.format(output_path))
    if csv_path is not None:
        examples.to_csv(csv_path, index=None)
        print('Successfully created the CSV file: {}'.format(csv_path))

if __name__ == '__main__':
    tf.app.run()
```

train 및 test 이미지에 위의 코드를 사용하여 각각 train.tfrecord 및 test.tfrecord를 변경하여

```
xml_dir = ‘images/test’
image_dir = ‘images/test’
output_path = 'annotations/test.record'
```

# tfrecord 생성

https://github.com/SIAnalytics/simplified_rbox_cnn/blob/master/create_dataset.py

필요한 부분 수정한 **create_dataset.py**

In [ ]:
!ln -s /content/drive/Shareddrives/OD/create_dataset.py .

In [ ]:
%cd /content

/content/workspace


In [ ]:
!python create_dataset.py --src_dir=/content/drive/Shareddrives/OD/data_real/train/ \
                                    --dst_path=./train.tfrecord \
                                    --is_include_only_pos

100% 1748/1748 [22:01<00:00,  1.32it/s]
N of TFRecords: 6506


이하 val 에 대해 반복

In [ ]:
import os
label_path = '/content/drive/Shareddrives/OD/data_real/val_etc/labels/'
files = os.listdir(label_path)
print('json 파일 총 개수 :', len(files))

json 파일 총 개수 : 220


In [ ]:
import json

all_json = None

for file in files :
    json_data = json.load(open(label_path + file))

    if all_json == None: 
        all_json = json_data
    else: 
        all_json['features'].extend(json_data['features'])

In [ ]:
with open('/content/drive/Shareddrives/OD/data_real/val_etc/labels/labels.json', 'w') as outfile:
    json.dump(all_json, outfile, indent='\t')

In [ ]:
! cd /content/drive/Shareddrives/OD/data_real/val_etc/labels && mv labels.json /content/drive/Shareddrives/OD/data_real/val

In [ ]:
!python create_dataset.py --src_dir=/content/drive/Shareddrives/OD/data_real/val/ \
                                    --dst_path=./val.tfrecord \
                                    --is_include_only_pos

100% 220/220 [02:18<00:00,  1.59it/s]
N of TFRecords: 822


# 사전 훈련 된 모델 다운로드
Tensorflow Model Zoo 에서 다운로드 할 준비가 된 많은 모델이 있습니다 .

일부 모델은 물체 감지 용이 아니므로 사용할 모델을 선택하는 데주의하십시오. 이 자습서에서는 다음 모델을 사용합니다.

SSD MobileNet V2 FPNLite 320x320 .

Colab Notebook에 다운로드하고 다음을 실행하여 압축을 풉니 다.

```
%cd pre-trained-models
!curl "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz" --output "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"
model_name = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
model_file = model_name + '.tar.gz'
tar = tarfile.open(model_file)
tar.extractall()
tar.close()
os.remove(model_file)
```

```
workspace/
├─ models/
│  ├─ community/
│  ├─ official/
│  ├─ orbit/
│  ├─ research/
│  ├─ my_mobilenet/
│  └─ ...
├─ annotations/
│  ├─ train/
│  └─ test/
├─ pre-trained-model/
├─ exported-models/
```

TF Object Detection API에서 모델 학습 및 평가에 필요한 모든 설정 및 필수 정보는 pipeline.config 파일에 있습니다.

살펴 보겠습니다.

우리가 변경해야 할 가장 중요한 것은
```
batch_size: 128
fine_tune_checkpoint: "PATH_TO_BE_CONFIGURED"
num_steps: 50000
num_classes: 2
fine_tune_checkpoint_type: "classification"
train_input_reader {   
  label_map_path: "PATH_TO_BE_CONFIGURED"   
  tf_record_input_reader {     
    input_path: "PATH_TO_BE_CONFIGURED"   
  } 
} 
eval_input_reader {   
  label_map_path: "PATH_TO_BE_CONFIGURED"   
  shuffle: false   
  num_epochs: 1   
  tf_record_input_reader {     
    input_path: "PATH_TO_BE_CONFIGURED"   
  }
}
```

실제 변경한 내용

`modified_pipeline.config` 로 저장

```
# For Fresh Training
fine_tune_checkpoint: "pre-trained-model/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0"
# For Contuining the Training
fine_tune_checkpoint: "exported_models/your_latest_batch/checkpoint/ckpt-0"
batch_size = 8 # Increase/Decrease this value depending on how fast your train job runs and the availability of the Compute Resources.
num_steps: 25000 # 25000 is a good number of steps to get a good loss.
fine_tune_checkpoint_type: "detection" # Set this to detection
train_input_reader {   
  label_map_path: "annotations/label_map.pbtxt"   # Set to location of label map
  tf_record_input_reader {     
    input_path: "annotations/train.tfrecord"   # Set to location of train TFRecord file
  } 
}
# Similarly do the same for the eval input reader
eval_input_reader {   
  label_map_path: "annotations/label_map.pbtxt"   
  shuffle: false   
  num_epochs: 1   
  tf_record_input_reader {     
    input_path: "annotations/test.tfrecord"   
  }
}
```

StackOverflow 에 대한 좋은 제안 은 다음과 같습니다.

**최대 배치 크기 = 사용 가능한 GPU 메모리 바이트 / 4 / (텐서 크기 + 학습 가능한 매개 변수)**

**fine_tune_checkpoint** 는 마지막으로 훈련 된 체크 포인트입니다 (체크 포인트는 Tensorflow 에서 모델을 저장하는 방법입니다).

처음으로 훈련을 시작하는 경우 이를 사전 훈련 된 모델로 설정하십시오.

이전에 훈련 된 체크 포인트에서 훈련을 계속하려면 해당 체크 포인트 경로로 설정하십시오. (이것은 처음부터 시작하는 대신 기능과 손실을 기반으로 교육을 계속할 것입니다).